# Reading and plotting tables

This notebook provides a basic overview of reading and manipulating tables using the `astropy` package. Questions, comments, and suggestions to the author are welcome. 

1. [Reading in Data](#reading-in-data)
1. [Accessing Rows and Columns](#accessing-rows-and-columns)
1. [Adding and Removing Information](#adding-and-removing-information)
1. [Using Conditions](#using-conditions)
1. [Joining Tables](#joining-tables)


## Reading in Data

The `astropy` package makes reading data from file very straight forward. Lets start by reading in a csv file and printing out the first ten rows of the table:

In [ ]:
from astropy.table import Table

sn_table = Table.read('sn1a_list.csv', format='ascii.csv')
print(sn_table)

When we use print to see a table, what we get in return isn't very pretty, or even very useful to look at. Fortunately,` astropy` tables have a special method called `show_in_notebook` that lets us visualize tables in a cleaner way. This method allows us to control how many entries we see at once, and even search for entries in the table. However, ss you might expect from the name, this method only works when using python in a Jupyter notebook.

Try interacting with the following table:

In [ ]:
sn_table.show_in_notebook(display_length=8)

Notice that in the above example there was no need to explicitly state the format of the file we are reading. This is because `Table.read()` will automatically iterate through multiple possible formats and try to find one that works. Unfortunately, this doesn't always work, and sometimes we need to provide a little more information.

This repository contains some data from the AMUSING survey as an ascii text files. You are encouraged to open this file using a text editor if you don't already know what an ascii data table looks like.

The `ascii` format is not one of the formats that `Table.read` will try automatically. If we try to read the table in the same way as the cell above we get an error, demonstrated in the following cell. If `astropy` cannot read a file, it will suggest a few formats that you can try. However, if you ever need to look up the format table, it’s easier to look it up online [here](http://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.read).


In [ ]:
# The following line is expected to raise an error
amusing_data = Table.read('amusing_data.txt')

Lets try reading some ascii tables the right way. Notice that we use the descriptive variable name `amusing_data` so that we know what kind of data is in the table. This is always a good practice, especially when dealing with multiple tables.

In [ ]:
# The following line should not raise an error
amusing_data = Table.read('amusing_data.txt', format='ascii')
amusing_data.show_in_notebook(display_length=8)

If you have trouble reading a table, sometimes its a quick and easy fix to open the file in a text editor and change it. Sometimes it’s a simple matter of removing an extra character or two at the beginning of the file. However, not all of the files you deal with will be stored in a human readable format. In fact, as astronomers you may find that most of the files you work with cannot be changed in this way. Because of this you should always try finding the right combination of arguments for `Table.read` first. A full list of your available options can be found [here](http://docs.astropy.org/en/v0.2.1/io/ascii/read.html).

## Accessing Rows and Columns

One of the biggest benefits of using `astropy` tables is how easy they make it to access and modify information. `astropy` lets us access both the rows and columns of a table using ***indexing***.

First let’s try accessing some columns in our table `sn_table`. Columns can be accessed by indexing the table using the desired column name as a string. Doing so will return a new table with only the columns we selected. To make life even easier, we can index a table using as many column names as we want:


In [ ]:
print("Fist lets check what the column names are:")
print(sn_table.colnames)

print("\n\nHeres a new table with only the 'sn' column of sn_table:\n")
sn_only_table = sn_table['sn']
print(sn_only_table)

print("\n\nWe can also select multiple columns at once:\n")
multiple_colum_table = sn_table['sn', 'ra', 'dec']
print(multiple_colum_table)


We can also access one or more rows from a table. However, unlike accessing columns, for rows we use a numerical index: 

In [ ]:
print("Here is the first row of sn_table:\n")
print(sn_table[0])  # Don't forget that indices in Python start at 0!

print("\n\n\nHere is the first four rows of sn_table:\n")
print(sn_table[0:5])


Notice that when accessing multiple rows, we use a colon. When we index an object like this in Python its called ***slicing***. In the example above, the slice `0:5` corresponds to indices 0 through 4. 

It should be noted that it doesn't matter in what order you access rows and columns from a table.


In [ ]:
print("The first row of the 'sn' column:\n")
print(sn_table['sn'][0])

print("\n\nThe 'sn' column of the first row. Notice that this is the same as before:\n")
print(sn_table[0]['sn'])

print("\n\nA few columns with only the first four rows:\n")
print(sn_table[0:5]['sn', 'ra', 'dec'])


## Adding and Removing Information

Adding and removing rows is pretty straight forward:

In [ ]:
# Lets make a copy of the table so that we can preserve the original
# We only do this so that the original can be used in a future example
example_table = sn_table.copy()

# Remove a single row by its index
example_table.remove_row(5)

# remove multiple rows by their index
sn_table.remove_rows([0, 1, 5])

# Append a new row to the end of the table
new_row = ['SN2009Y', 'NGC5728_1', '14:42:23.68', '-17:14:48.4',
           0.0093, 1.018, 0.014, 0.167, 0.0150, 13.981, 0.0440]
example_table.add_row(new_row)


Columns can be removed in much the same way:

In [ ]:
example_table = sn_table.copy()

# remove the column 'ST'
example_table.remove_column('ST')


One useful feature is the `keep_columns` method. This method removes all the columns from a table except ones that you specify:

In [ ]:
# Keep only the 'sn', 'ra', and 'dec' columns
example_table.keep_columns(['sn', 'ra', 'dec'])
print(example_table)


## Using Conditions

Let’s say we only want the entries in `amusing_data` where the `QF` column has the value `"S"`. It might be tempting to iterate over the table and remove any rows that don't have the value "S" in the column `QF`. Instead, the following is much simpler

In [ ]:
indicies = amusing_data['QF'] == "S"
data_we_want = amusing_data[indicies]
data_we_want.show_in_notebook(display_length=8)

If we want, we can even combine conditions using numpy:

In [ ]:
import numpy as np

# Get rows where QF is either S or X
s_indicies = amusing_data['QF'] == "S"
x_indicies = amusing_data['QF'] == "X"
s_or_x_indices = np.logical_or(s_indicies, x_indicies)

# Get rows where QF is S and Ha > 1e-13
ha_indices = amusing_data['Ha'] > (1 * 10 ** -13)
s_and_ha_indices = np.logical_and(s_indicies, ha_indices)


## Joining Tables

Combining two tables together is called ***joining*** them. This lets us combine information from multiple different tables together into a single, master table. Joining works by referencing ***key*** values. These are values that are used to match rows together so that we know what rows correspond to each other in different tables.

Imagine you have multiple tables of supernova data. A sensible key to pick when joining tables might be the supernova's name. Unfortunately, its very common for different data tables to use different naming strategies. The supernova "sn2012fr" might be stored in a table as "2012fr", "sn12fr", "2012fr", or even by the name of its host galaxy "NGC1365".

Here’s an example of how to combine the `sn_table` `amusing_data` tables:


In [ ]:
from astropy.table import join

amusing_data_copy = amusing_data.copy()
sn_table_copy = sn_table.copy()

# we give the key columns the same name
sn_table_copy.rename_column('amusing_name', 'amusing_key')
amusing_data_copy.rename_column('SNNAME', 'amusing_key')

combined_table = join(sn_table_copy, amusing_data_copy, keys='amusing_key')
combined_table.show_in_notebook(display_length=8)